# Beautiful Soup

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/warandpeace.html")
bsObj = BeautifulSoup(html)

/Users/michaelheydt/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/michaelheydt/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [2]:
nameList = bsObj.findAll("span", {"class":"green"})
for name in nameList:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


# Requests

In [3]:
# The number of librarian-related job positions that the federal government is currently hiring for
import requests
# via http://www.opm.gov/policy-data-oversight/classification-qualifications/general-schedule-qualification-standards/#url=List-by-Occupational-Series
LIBSERIES = 1410
resp = requests.get("https://data.usajobs.gov/api/jobs", params = {'series': LIBSERIES})
#print(resp.json()['TotalJobs'])
resp

<Response [404]>

In [4]:
# Total number of visitors to the White House in 2012
# landing page:
# https://www.whitehouse.gov/briefing-room/disclosures/visitor-records
import csv
import os
import requests
from shutil import unpack_archive
LOCAL_DATADIR = "/tmp/whvisitors"
url = 'https://www.whitehouse.gov/sites/default/files/disclosures/whitehouse-waves-2012.csv_.zip'
zname = os.path.join(LOCAL_DATADIR, os.path.basename(url))
cname = os.path.join(LOCAL_DATADIR, 'WhiteHouse-WAVES-2012.csv')
os.makedirs(LOCAL_DATADIR, exist_ok = True)

# Download the zip
if not os.path.exists(zname):
    print("Downloading", url, 'to', zname)
    z = requests.get(url).content
    with open(zname, 'wb') as f:
        f.write(z)

# unzip it
print("Unzipping", zname, 'to', LOCAL_DATADIR)
unpack_archive(zname, LOCAL_DATADIR)
# the file was zipped on a Mac, yet still uses Windows encoding...mkaaay
rows = list(csv.DictReader(open(cname, encoding = 'ISO-8859-1')))
print(len(rows))
# 934872

Unzipping /tmp/whvisitors/whitehouse-waves-2012.csv_.zip to /tmp/whvisitors


ReadError: /tmp/whvisitors/whitehouse-waves-2012.csv_.zip is not a zip file

## Florida's lists of executions

In [ ]:
# The total number of inmates executed by Florida since 1976
import requests
from lxml import html
url = "http://www.dc.state.fl.us/oth/deathrow/execlist.html"

doc = html.fromstring(requests.get(url).text)
tables = doc.cssselect('table.dcCSStableLight')
rows = tables[0].cssselect('tr')
# the first row is just the header row
print(len(rows) - 1)

# lxml

In [ ]:
# Number of days until Texas's next scheduled execution
from datetime import datetime
from lxml import html
import pytz
import requests
url = "http://www.tdcj.state.tx.us/death_row/dr_scheduled_executions.html"
# fetch and parse the page
doc = html.fromstring(requests.get(url).text)
# Get our time in central texas time; http://stackoverflow.com/a/22109768/160863
texas_time = pytz.timezone("US/Central")
today = texas_time.localize(datetime(*datetime.now().timetuple()[0:3])) # whatever, too lazy to look up the idiom
for row in doc.xpath('//table/tr')[1:]:
    # Even though this table is sorted in reverse-chronological order,
    #  sometimes the executions happen more quickly than the updates to the
    #  webpage, can't assume the first row is always the upcoming execution
    #
    # Each row looks like:
    # | 08/12/2015 |  Info | Lopez | Daniel | 999555 | 09/15/1987 | H | 03/16/2010 | Nueces |
    col = row.cssselect('td')[0]
    exdate = datetime.strptime(col.text_content(), '%m/%d/%Y')
    exdate = texas_time.localize(exdate)
    if (exdate >= today):
        print((exdate - today).days, "days")
        break